In [ ]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row, SparkSession
from pyspark.sql.functions import regexp_extract

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
accessLines = spark.readStream.text("logs")

In [ ]:
contentSizeExp = r'\s(\d+)$'
statusExp = r'\s(\d{3})\s'
generalExp = r'\"(\S+)\s(\S+)\s*(\S*)\"'
timeExp = r'\[(\d{2}/\w{3}/\d{4}:\d{2}:\d{2}:\d{2} -\d{4})]'
hostExp = r'(^\S+\.[\S+\.]+\S+)\s'

logsDF = accessLines.select(regexp_extract('value', hostExp, 1).alias('host'),
                         regexp_extract('value', timeExp, 1).alias('timestamp'),
                         regexp_extract('value', generalExp, 1).alias('method'),
                         regexp_extract('value', generalExp, 2).alias('endpoint'),
                         regexp_extract('value', generalExp, 3).alias('protocol'),
                         regexp_extract('value', statusExp, 1).cast('integer').alias('status'),
                         regexp_extract('value', contentSizeExp, 1).cast('integer').alias('content_size'))

In [ ]:
statusCountsDF = logsDF.groupBy(logsDF.status).count()

In [ ]:
query = ( statusCountsDF.writeStream.outputMode("complete").format("console")\
         .queryName("counts").start() )

In [ ]:
query.awaitTermination()

### Top URL's Count For Past 30 Seconds

In [ ]:
logsDF2 = logsDF.withColumn("eventTime", func.current_timestamp())

In [ ]:
endpointCounts = logsDF2.groupBy(func.window(func.col("eventTime"), \
      "30 seconds", "10 seconds"), func.col("endpoint")).count()

In [ ]:
sortedEndpointCounts = endpointCounts.orderBy(func.col("count").desc())

In [ ]:
query = sortedEndpointCounts.writeStream.outputMode("complete").format("console") \
      .queryName("counts").start()

In [ ]:
query.awaitTermination()